In [9]:
import pandas as pd
import requests, time, json
from datetime import datetime

from sqlalchemy import create_engine

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials from txt file
with open('../../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)

today = datetime.today().strftime('%Y-%m-%d')
sides = ['offensive', 'defensive']
lastNgames = "10"

league = "nba"
pymysql_conn_str = creds['pymysql'][league]

### TEAM OFF AND DEF PLAY TYPE RANKS

In [2]:
baseUrl = "https://www.nba.com/stats/teams/{playtype}?TypeGrouping={sideofball}"
playTypes = ['isolation', 'transition', 'ball-handler', 'roll-man',
            'playtype-post-up', 'spot-up', 'hand-off', 'cut', 'off-screen',
            'putbacks']

service = Service(r"..\..\browser\geckodriver.exe")
driver = webdriver.Firefox(service=service)
data = []

for s in sides:
    for play in playTypes:
        # get html page source data
        url = baseUrl.format(playtype=play, sideofball=s)
        driver.get(url)
        time.sleep(3)
        ps = driver.page_source
        soup = bs(ps)
        
        # extract table holding the data
        try:
            tables = soup.find_all('table')
            table = tables[0]
            
            #pull out col names
            headers = table.find('thead').find_all('th')
            cols = []
            for i in headers:
                cols.append(i.get_attribute_list('field')[0])
            cols.append('tid')
            cols.append('sob')
            cols.append('play')
            
            # pull out data from the body
            #data = []
            rows = table.find('tbody').find_all('tr')
            for i in rows:
                row = []
                rowData = i.find_all('td')
                for j in rowData:
                    row.append(j.text)
                    try:
                        tid = j.find('a').get_attribute_list('href')
                    except:
                        pass
                tid = tid[0].split('/')[3]
                row.append(tid)
                row.append(s)
                row.append(play)
                data.append(row)
        except:
            continue
                
# combine headers with data in a dataframe        
df = pd.DataFrame(data, columns=cols)


driver.close()

#sides playTypes
cols.append('scoreFreqRank')
ranked = pd.DataFrame(columns = cols)
for s in sides:
    for pt in playTypes:
        temp = df[(df['sob'] == s) & (df['play'] == pt)]
        temp.loc[:,'scoreFreqRank'] = temp.loc[:,'SCORE_POSS_PCT'].rank(ascending=False)
        ranked = pd.concat([ranked, temp])

ranked.loc[:,'date'] = today

#saving data
ranked.to_csv('../../data/' + today + '_teamPlayTypes.csv', index=False)
ranked.to_sql(name="statsteamplaytypes", con=pymysql_conn_str, if_exists="append", index=False)


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\3560502544.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,'scoreFreqRank'] = temp.loc[:,'SCORE_POSS_PCT'].rank(ascending=False)
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\3560502544.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ranked = pd.concat([ranked, temp])
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\3560502544.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## PLAYER OFF PLAY TYPE STATS

In [3]:
baseUrl = "https://www.nba.com/stats/players/{playtype}?TypeGrouping={sideofball}"
service = Service(r"..\..\browser\geckodriver.exe")
driver = webdriver.Firefox(service=service)
data = []
buttonXpath = "/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]"

playTypes = ['isolation', 'transition', 'ball-handler', 'roll-man',
            'playtype-post-up', 'spot-up', 'hand-off', 'cut', 'off-screen',
            'putbacks']

for play in playTypes:
    # get html page source data
    url = baseUrl.format(playtype=play, sideofball="offensive")
    
    # get html page source data
    driver.get(url)
    #time.sleep(4)
    
    ## need to open page then change filter from 1 to -1 so that all players load
    ## div class="Crom_cromSettings__ak6Hd"  > 
    #####select class="DropDown_select__4pIg9"
    #######option value=-1
    try:
        WebDriverWait(driver, timeout=20).until(lambda d: d.find_element("xpath", buttonXpath))
        pagenationFilter = driver.find_element("xpath", buttonXpath)
        pagenationFilter.click()
    except:
        pagenationFilter = driver.find_element("xpath", buttonXpath)
        driver.execute_script("arguments[0].click();", pagenationFilter)
    
    time.sleep(1)
    ps = driver.page_source
    soup = bs(ps)

        
    # extract table holding the data
    try:
        tables = soup.find_all('table')
        table = tables[0]
        
        #pull out col names
        headers = table.find('thead').find_all('th')
        cols = []
        for i in headers:
            cols.append(i.get_attribute_list('field')[0])
        # add column names that will be created
        cols.append('pid')
        cols.append('tid')
        cols.append('sob')
        cols.append('play')
        
        # pull out data from the body
        #data = []
        
        # extract each row
        rows = table.find('tbody').find_all('tr')
        for i in rows:
            row = []
        
            # extract each piece of data from a single row
            rowData = i.find_all('td')
            for j in rowData:
                row.append(j.text)
        
                try:
                    href = j.find('a').get_attribute_list('href')
                    #assigning href to correct variable for later processing.
                    if "player" in href[0]:
                        pid = href[0].split('/')[3]
                        
                    else:
                        tid = href[0].split('/')[3]      
                except:
                    pass
            
            #add additional data to the row before it is added to the agg'ed data++
            row.append(pid)
            row.append(tid)
            row.append("offensive")
            row.append(play)
        
            # add the gathered row data into the master list that will be converted to dataframe
            data.append(row)
    except Exception as e:
        print(e)
        continue
            
# combine headers with data in a dataframe        
df = pd.DataFrame(data, columns=cols)

driver.close()

headers = table.find('thead').find_all('th')
cols = []
for i in headers:
    cols.append(i.get_attribute_list('field')[0])
# add column names that will be created
cols.append('pid')
cols.append('tid')
cols.append('sob')
cols.append('play')

#sides playTypes
cols.append('freqRank')
cols.append('scoreFreqRank')
ranked = pd.DataFrame(columns = cols)
ranked

for pt in playTypes:
    temp = df[(df['sob'] == "offensive") & (df['play'] == pt)]
    temp.loc[:,'scoreFreqRank'] = temp.loc[:,'SCORE_POSS_PCT'].rank(ascending=False)
    temp.loc[:,'freqRank'] = temp.loc[:,'POSS_PCT'].rank(ascending=False)
    ranked = pd.concat([ranked, temp])

ranked.loc[:,'date'] = today

#save data
ranked.to_csv('../../data/' + today + '_playerPlayTypes.csv', index=False)
ranked.to_sql(name="statsplayerplaytypes", con=pymysql_conn_str, if_exists="append", index=False)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\2486467319.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,'scoreFreqRank'] = temp.loc[:,'SCORE_POSS_PCT'].rank(ascending=False)
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\2486467319.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,'freqRank'] = temp.loc[:,'POSS_PCT'].rank(ascending=False)
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_16456\2486467319.py:113: FutureWarning: The behavior of DataFrame concatenation with em

### TEAM OFF AND DEF SHOOTING BY ZONE

In [4]:
#col names
colsZone = ['TEAM_NAME',
 'paintRaFgm', 'paintRaFga', 'paintRaFgPct',
 'paintNoRaFgm','paintNoRaFga','paintNoRaFgPct',
 'midFgm','midFga','midFgPct',
 'leftC3Fgm','leftC3Fga','leftC3FgPct',
 'rightC3Fgm','rightC3Fga','rightC3FgPct',
 'bothC3Fgm','bothC3Fga','bothC3FgPct',
 'aboveBreak3Fgm','aboveBreak3Fga','aboveBreak3FgPct',
 'tid','sob']

service = Service(r"..\..\browser\geckodriver.exe")
driver = webdriver.Firefox(service=service)

data = []
for s in sides:
    if s == 'offensive':
        url = 'https://www.nba.com/stats/teams/shooting?DistanceRange=By+Zone&LastNGames=' + lastNgames
    else:
        url = 'https://www.nba.com/stats/teams/opponent-shooting?DistanceRange=By+Zone&LastNGames=' + lastNgames
        
    # get html page source data
    driver.get(url)
    time.sleep(3)
    ps = driver.page_source
    soup = bs(ps)
    
    # extract table holding the data
    tables = soup.find_all('table')
    table = tables[-1]
      
    # pull out data from the body
    #data = []
    rows = table.find('tbody').find_all('tr')
    for i in rows:
        row = []
        rowData = i.find_all('td')
        for j in rowData:
            # convert number strings to numeric 
            try:
                row.append(pd.to_numeric(j.text))
            except:
                row.append(j.text)
                
            # grab team id from the one td that has it
            try:
                tid = j.find('a').get_attribute_list('href')
            except:
                pass
        tid = tid[0].split('/')[3]
        row.append(tid)
        row.append(s)
        data.append(row)

driver.close()

# combine headers with data in a dataframe        
dfZoneShooting = pd.DataFrame(data, columns=colsZone)

# calculate all points in the paint restricted area + non-ra
dfZoneShooting.loc[:,'paintAllFgm'] =  dfZoneShooting['paintRaFgm'] + dfZoneShooting['paintNoRaFgm']
dfZoneShooting.loc[:,'paintAllFga'] =  dfZoneShooting['paintRaFga'] + dfZoneShooting['paintNoRaFga']
dfZoneShooting.loc[:,'paintAllFgPct'] =  round((dfZoneShooting['paintAllFgm'] / dfZoneShooting['paintAllFga']) * 100,3)

dfZoneShooting.loc[:,'date'] = today

#save data
dfZoneShooting.to_csv('../../data/' + today + '_teamShotZones.csv', index=False)
dfZoneShooting.to_sql(name="statsteamshotzones", con=pymysql_conn_str, if_exists="append", index=False)

### OFF PLAYER SHOT ZONES

In [5]:
colsZoneP = ['PLAYER_NAME', 'TEAM', 'AGE',
 'paintRaFgm', 'paintRaFga', 'paintRaFgPct',
 'paintNoRaFgm','paintNoRaFga','paintNoRaFgPct',
 'midFgm','midFga','midFgPct',
 'leftC3Fgm','leftC3Fga','leftC3FgPct',
 'rightC3Fgm','rightC3Fga','rightC3FgPct',
 'bothC3Fgm','bothC3Fga','bothC3FgPct',
 'aboveBreak3Fgm','aboveBreak3Fga','aboveBreak3FgPct',
 'pid','tid']

service = Service(r"..\..\browser\geckodriver.exe")
driver = webdriver.Firefox(service=service)

data = []
url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames=' + lastNgames
    
# get html page source data
driver.get(url)
time.sleep(3)

## need to open page then change filter from 1 to -1 so that all players load
## div class="Crom_cromSettings__ak6Hd"  > 
#####select class="DropDown_select__4pIg9"
#######option value=-1
try:
    WebDriverWait(driver, timeout=20).until(lambda d: d.find_element("xpath", buttonXpath))
    pagenationFilter = driver.find_element("xpath", buttonXpath)
    pagenationFilter.click()
except:
    pagenationFilter = driver.find_element("xpath", buttonXpath)
    driver.execute_script("arguments[0].click();", pagenationFilter)

time.sleep(1)
ps = driver.page_source
soup = bs(ps)

# extract table holding the data
tables = soup.find_all('table')
table = tables[-1]
  
# pull out data from the body
data = []
rows = table.find('tbody').find_all('tr')
for i in rows:
    row = []
    rowData = i.find_all('td')
    for j in rowData:
        # convert number strings to numeric 
        try:
            if j.text == '-':
                txt = 0
            else:
                txt = j.text
            row.append(pd.to_numeric(txt))
        except:
            row.append(j.text)
            
        # grab team id and player id from the one td that has it
        try:
            href = j.find('a').get_attribute_list('href')[0]
            if 'player' in href:
                pid = href.split('/')[3]
            if 'team' in href:
                tid = href.split('/')[3]
            
        except:
            pass
    
    row.append(pid)
    row.append(tid)
    data.append(row)

driver.close()

# combine headers with data in a dataframe        
dfpZoneShooting = pd.DataFrame(data, columns=colsZoneP)

# calculate all points in the paint restricted area + non-ra
dfpZoneShooting.loc[:,'paintAllFgm'] =  dfpZoneShooting['paintRaFgm'] + dfpZoneShooting['paintNoRaFgm']
dfpZoneShooting.loc[:,'paintAllFga'] =  dfpZoneShooting['paintRaFga'] + dfpZoneShooting['paintNoRaFga']
dfpZoneShooting.loc[:,'paintAllFgPct'] =  round(dfpZoneShooting['paintAllFgm'] / dfpZoneShooting['paintAllFga']) * 100,3)

dfpZoneShooting.loc[:,'date'] = today

#save data
dfpZoneShooting.to_csv('../../data/' + today + '_playerShotZones.csv', index=False)
dfpZoneShooting.to_sql(name="statsplayershotzones", con=pymysql_conn_str, if_exists="append", index=False)

### TEAM DEF RTG and PLAYER DEF RTG

In [ ]:
playerDefRatingUrl = 'https://www.nba.com/stats/players/advanced?CF=MIN*GE*15&SeasonType=Regular%20Season&dir=-1&sort=DEF_RATING'
teamDefRatingUrl = 'https://www.nba.com/stats/teams/advanced?LastNGames=10&dir=A&sort=DEF_RATING'
                    